In [1]:
from ollama import chat
from ollama import ChatResponse
import pandas as pd
import numpy as np
import json
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.tests.test_metaestimators_metadata_routing import classes_multi
from tqdm import tqdm
from src.simplify import classif, simplify
from src.utils_data import load_data, all_annotation
from src.simplify import classe2CECR, CECR2classe, init_classes
from src.indentify_local import labels, definitions

In [1]:
### dataset verifications

In [3]:
with open('%s/%s' %('.', 'annotations.json'), 'r') as f:
    json_data = json.load(f)
local_df = pd.DataFrame(json_data)

In [4]:
local_df

,text,annotations
0,Mon arrivée en France Je suis venue en Fran...,"[{'text': 'rejoindre', 'start': 71, 'end': 80,..."
1,Les six scénarios qui marchent au cinéma. 1...,"[{'text': 'veut sa peau', 'start': 278, 'end':..."
2,J'ai tardé à t'écrire. J'étais trop occupé ces...,"[{'text': 'S'éloignent', 'start': 102, 'end': ..."
3,PLANNING DES TRAVAUX D’OCTOBRE 2023 À JUILL...,"[{'text': 'flux', 'start': 68, 'end': 72, 'lab..."
4,Quand vivre ailleurs pendant quelques jours ou...,"[{'text': 'L’hébergement temporaire', 'start':..."
...,...,...
423,"La Tunisie de Rajah, militante pour les droits...","[{'text': 'discriminations raciales', 'start':..."
424,ESSAYEZ UN NOUVEAU SPORT CET ÉTÉ ! ULTIMATE...,[{'text': 'par exemple sur les immenses plages...
425,LES ROSIERS GRIMPANTS Ils embellissent nos ...,"[{'text': 'embellissent', 'start': 29, 'end': ..."
426,Histoire de l'aviation 1. Depuis la nuit d...,"[{'text': 'biplan', 'start': 386, 'end': 392, ..."


In [9]:
new = pd.read_excel('annotations_completes.xlsx')

In [11]:
new.TermLabel.unique()

array(['Mot difficile ou inconnu', 'Graphie, problème de déchiffrage',
       'Ordre syntaxique inhabituel',
       'Figure de style, expression idiomatique',
       'Référence culturelle difficile', 'Difficulté liée à la grammaire',
       "Trop d'informations secondaires",
       'Indice de cohésion difficile (connecteur, pronom, inférence)',
       'Autre', 'Mot inconnu', 'Problème de cohésion'], dtype=object)

In [19]:
new['annotations'] = new.apply(lambda row: {'text': row['AnnotatedTerm'], 'label': row['TermLabel']}, axis=1)
grouped = new.groupby('Text')['annotations'].apply(list).reset_index()
local_df = grouped.rename(columns={'Text': 'term'})

In [20]:
local_df

,term,annotations
0,#Episode1 : C'est parti !! La première rencont...,"[{'text': '#Episode1', 'label': 'Graphie, prob..."
1,"- Lavage avec action mécanique, rinçage et ess...","[{'text': 'action mécanique', 'label': 'Mot di..."
2,"09 janvier Stretching #1 LE STRETCHING, ON...","[{'text': 'Inspirez', 'label': 'Mot difficile ..."
3,1 NUITÉE GRATUITE À OOSTENDE - Rien ne pres...,"[{'text': 'Mercator,', 'label': 'Référence cul..."
4,10 janvier FOOD Tinto verano Idéal pour u...,"[{'text': 'Kcal:', 'label': 'Référence culture..."
...,...,...
454,Étudiants en Europe: bons plans pour se loger...,"[{'text': 'où', 'label': 'Difficulté liée à la..."
455,Évènements nationaux Les évènement nationa...,"[{'text': 'Évènements nationaux', 'label': 'Gr..."
456,"Êtes-vous tendance ? 1. Pour vous, un logem...","[{'text': 'tissus ethniques.', 'label': 'Mot d..."
457,électricien bâtiment VMA LOUVAIN LA NEUVE R...,"[{'text': 'câbles,', 'label': 'Mot difficile o..."


In [14]:
local_df.iloc[450]['annotations']

[{'text': 'collectif', 'label': 'Mot difficile ou inconnu'},
 {'text': 'faciès.', 'label': 'Mot difficile ou inconnu'},
 {'text': 'faciès.', 'label': 'Mot difficile ou inconnu'},
 {'text': 'faciès.', 'label': 'Mot difficile ou inconnu'},
 {'text': 'antiraciste.', 'label': 'Graphie, problème de déchiffrage'},
 {'text': "elle ne s'en donne pas les moyens.",
  'label': 'Ordre syntaxique inhabituel'},
 {'text': 'moyens.', 'label': 'Graphie, problème de déchiffrage'},
 {'text': 'actions?', 'label': 'Graphie, problème de déchiffrage'},
 {'text': 'syndrome', 'label': 'Mot difficile ou inconnu'},
 {'text': 'syndrome', 'label': 'Graphie, problème de déchiffrage'},
 {'text': 'syndrome Tarzan.', 'label': 'Mot difficile ou inconnu'},
 {'text': "d'incitation", 'label': 'Mot difficile ou inconnu'},
 {'text': "d'incitation à la haine raciale.",
  'label': 'Mot difficile ou inconnu'},
 {'text': 'enchaîné', 'label': 'Mot difficile ou inconnu'},
 {'text': 'enchaîné', 'label': 'Figure de style, expressio

In [6]:
new.groupby('Text')

In [8]:
new = new.rename(columns={'AnnotatedTerm': 'text', 'TermLabel':'label'})
new

,Username,FormID,Text,TextIndex,text,TermIndex,label,AdditionnalInfo
0,VAV,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,collectif,10,Mot difficile ou inconnu,NaN
1,VAV,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN
2,VIL,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN
3,EMD,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN
4,VIL,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,antiraciste.,11,"Graphie, problème de déchiffrage",NaN
...,...,...,...,...,...,...,...,...
10268,VAV,14_2,Recette pour faire de l’eau aromatisée Maté...,16,Ingrédients,35,"Graphie, problème de déchiffrage",NaN
10269,VIL,14_2,Recette pour faire de l’eau aromatisée Maté...,16,Ingrédients,35,Mot difficile ou inconnu,NaN
10270,VAV,14_2,Recette pour faire de l’eau aromatisée Maté...,16,brin,44,Mot difficile ou inconnu,NaN
10271,VIL,14_2,Recette pour faire de l’eau aromatisée Maté...,16,millilitres,48,Mot difficile ou inconnu,NaN


In [8]:
new.columns

Index(['Username', 'FormID', 'Text', 'TextIndex', 'AnnotatedTerm', 'TermIndex',
       'TermLabel', 'AdditionnalInfo'],
      dtype='object')

In [11]:
new['text']

0           collectif
1             faciès.
2             faciès.
3             faciès.
4        antiraciste.
             ...     
10268     Ingrédients
10269     Ingrédients
10270            brin
10271     millilitres
10272      aromatisée
Name: text, Length: 10273, dtype: object

In [5]:
global_df, local_df = load_data(file_path='.', global_file='Qualtrics_Annotations_B.csv', local_file='annotations_completes.xlsx')
local_df.classe.unique()

Index(['text', 'annotations'], dtype='object')


array(['B1', 'A2', 'A1', 'B2'], dtype=object)

In [7]:
local_df

,text,annotations,text_indice,gold_score_20_label,classe
0,#Episode1 : C'est parti !! La première rencont...,"[{'text': '#Episode1', 'label': 'Graphie, prob...",229,Accessible,B1
1,"- Lavage avec action mécanique, rinçage et ess...","[{'text': 'action mécanique', 'label': 'Mot di...",1061,Accessible,B1
2,"09 janvier Stretching #1 LE STRETCHING, ON...","[{'text': 'Inspirez', 'label': 'Mot difficile ...",2099,Facile,A2
3,1 NUITÉE GRATUITE À OOSTENDE - Rien ne pres...,"[{'text': 'Mercator,', 'label': 'Référence cul...",1155,Facile,A2
4,10 janvier FOOD Tinto verano Idéal pour u...,"[{'text': 'Kcal:', 'label': 'Référence culture...",2107,Facile,A2
...,...,...,...,...,...
454,Étudiants en Europe: bons plans pour se loger...,"[{'text': 'où', 'label': 'Difficulté liée à la...",190,Accessible,B1
455,Évènements nationaux Les évènement nationa...,"[{'text': 'Évènements nationaux', 'label': 'Gr...",1477,Facile,A2
456,"Êtes-vous tendance ? 1. Pour vous, un logem...","[{'text': 'tissus ethniques.', 'label': 'Mot d...",7,Facile,A2
457,électricien bâtiment VMA LOUVAIN LA NEUVE R...,"[{'text': 'câbles,', 'label': 'Mot difficile o...",1568,Facile,A2


In [6]:
global_df.classe

text_indice
422     B1
1735    A2
2084    A2
438     A2
133     B1
        ..
1181    A2
942     A2
172     A2
664     B1
1763    A2
Name: classe, Length: 461, dtype: object

In [17]:
global_df, local_df = load_data(file_path='.', global_file='Qualtrics_Annotations_B.csv', local_file='annotations.json')
local_df

Index(['text', 'annotations'], dtype='object')


,text,annotations,text_indice,gold_score_20_label,classe
0,Mon arrivée en France Je suis venue en Fran...,"[{'text': 'rejoindre', 'start': 71, 'end': 80,...",1957,Facile,A2
1,Les six scénarios qui marchent au cinéma. 1...,"[{'text': 'veut sa peau', 'start': 278, 'end':...",165,Accessible,B1
2,J'ai tardé à t'écrire. J'étais trop occupé ces...,"[{'text': 'S'éloignent', 'start': 102, 'end': ...",486,Accessible,B1
3,PLANNING DES TRAVAUX D’OCTOBRE 2023 À JUILL...,"[{'text': 'flux', 'start': 68, 'end': 72, 'lab...",1043,Facile,A2
4,Quand vivre ailleurs pendant quelques jours ou...,"[{'text': 'L’hébergement temporaire', 'start':...",1605,Facile,A2
...,...,...,...,...,...
423,"La Tunisie de Rajah, militante pour les droits...","[{'text': 'discriminations raciales', 'start':...",77,Facile,A2
424,ESSAYEZ UN NOUVEAU SPORT CET ÉTÉ ! ULTIMATE...,[{'text': 'par exemple sur les immenses plages...,159,Facile,A2
425,LES ROSIERS GRIMPANTS Ils embellissent nos ...,"[{'text': 'embellissent', 'start': 29, 'end': ...",1078,Facile,A2
426,Histoire de l'aviation 1. Depuis la nuit d...,"[{'text': 'biplan', 'start': 386, 'end': 392, ...",1723,Facile,A2


In [16]:
local_df

,Username,FormID,Text,TextIndex,text,TermIndex,label,AdditionnalInfo,text_indice,gold_score_20_label,classe
0,VAV,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,collectif,10,Mot difficile ou inconnu,NaN,-1,None,NaN
1,VAV,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN,-1,None,NaN
2,VIL,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN,-1,None,NaN
3,EMD,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,faciès.,101,Mot difficile ou inconnu,NaN,-1,None,NaN
4,VIL,1_1,« Sortir du syndrome Tarzan » La brigade an...,1,antiraciste.,11,"Graphie, problème de déchiffrage",NaN,-1,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10268,VAV,14_2,Recette pour faire de l’eau aromatisée Maté...,16,Ingrédients,35,"Graphie, problème de déchiffrage",NaN,-1,None,NaN
10269,VIL,14_2,Recette pour faire de l’eau aromatisée Maté...,16,Ingrédients,35,Mot difficile ou inconnu,NaN,-1,None,NaN
10270,VAV,14_2,Recette pour faire de l’eau aromatisée Maté...,16,brin,44,Mot difficile ou inconnu,NaN,-1,None,NaN
10271,VIL,14_2,Recette pour faire de l’eau aromatisée Maté...,16,millilitres,48,Mot difficile ou inconnu,NaN,-1,None,NaN


In [12]:
global_df, local_df = load_data('.')
df_all_annotations = all_annotation(local_df)

In [9]:
local_df

,text,annotations
0,Mon arrivée en France Je suis venue en Fran...,"[{'text': 'rejoindre', 'start': 71, 'end': 80,..."
1,Les six scénarios qui marchent au cinéma. 1...,"[{'text': 'veut sa peau', 'start': 278, 'end':..."
2,J'ai tardé à t'écrire. J'étais trop occupé ces...,"[{'text': 'S'éloignent', 'start': 102, 'end': ..."
3,PLANNING DES TRAVAUX D’OCTOBRE 2023 À JUILL...,"[{'text': 'flux', 'start': 68, 'end': 72, 'lab..."
4,Quand vivre ailleurs pendant quelques jours ou...,"[{'text': 'L’hébergement temporaire', 'start':..."
...,...,...
423,"La Tunisie de Rajah, militante pour les droits...","[{'text': 'discriminations raciales', 'start':..."
424,ESSAYEZ UN NOUVEAU SPORT CET ÉTÉ ! ULTIMATE...,[{'text': 'par exemple sur les immenses plages...
425,LES ROSIERS GRIMPANTS Ils embellissent nos ...,"[{'text': 'embellissent', 'start': 29, 'end': ..."
426,Histoire de l'aviation 1. Depuis la nuit d...,"[{'text': 'biplan', 'start': 386, 'end': 392, ..."


In [14]:
local_df.loc[0].annotations

[{'text': 'rejoindre',
  'start': 71,
  'end': 80,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV', 'VIL'],
  'confidence': 0.67},
 {'text': 'Heureusement',
  'start': 308,
  'end': 320,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': "d'alphabétisation.",
  'start': 968,
  'end': 986,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV', 'VIL'],
  'confidence': 0.67},
 {'text': "et j'ai à nouveau perdu tous mes amis.",
  'start': 579,
  'end': 617,
  'label': "Trop d'informations secondaires",
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': "et qui m'a appris à me débrouiller en France.",
  'start': 704,
  'end': 749,
  'label': "Trop d'informations secondaires",
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': 'connaissais',
  'start': 218,
  'end': 229,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV'],
  'confidence': 0.33},
 {'text': 'étaient',
  

In [4]:
local_df = new

In [5]:
indexes = []
for text_indice, global_text in global_df.iterrows():
    matching_indexes = local_df[local_df['Text'] == global_text['text']].index.tolist()
    if not matching_indexes:
        print(text_indice)
    indexes.append(matching_indexes[0] if matching_indexes else -1)

1213
1448


In [18]:
global_df['text']

text_indice
422     « Sortir du syndrome Tarzan »    La brigade an...
1735    Les cultures en Afrique du Nord    Les trois p...
2084    Bienvenue à Gardanne    Gardanne est une ville...
438     Le 8 octobre c’est la première journée paralym...
133     Une destination à conseiller pour les vacances...
                              ...                        
1181    Démonstration Ikebana    1 variation et 1 créa...
942     Une drôle de journée    Et me voilà repartie p...
172     L'adolescence ailleurs    En Amérique, en Asie...
664     Je l'aime à mourir    Moi je n'étais rien  Et ...
1763    Recette pour faire de l’eau aromatisée    Maté...
Name: text, Length: 461, dtype: object

In [17]:
local_df['Text']]

0        « Sortir du syndrome Tarzan »    La brigade an...
1        « Sortir du syndrome Tarzan »    La brigade an...
2        « Sortir du syndrome Tarzan »    La brigade an...
3        « Sortir du syndrome Tarzan »    La brigade an...
4        « Sortir du syndrome Tarzan »    La brigade an...
                               ...                        
10268    Recette pour faire de l’eau aromatisée    Maté...
10269    Recette pour faire de l’eau aromatisée    Maté...
10270    Recette pour faire de l’eau aromatisée    Maté...
10271    Recette pour faire de l’eau aromatisée    Maté...
10272    Recette pour faire de l’eau aromatisée    Maté...
Name: Text, Length: 10273, dtype: object

In [8]:
indexes = []
for i, local_text in local_df.iterrows():
    if local_text['Text'] not in global_df['text'].values:
        indexes.append(i)
        print(i)

In [12]:
indexes.count(-1)

2

In [13]:
indexes

[0,
 37,
 59,
 81,
 109,
 160,
 187,
 218,
 239,
 279,
 320,
 352,
 367,
 411,
 417,
 429,
 448,
 471,
 477,
 519,
 555,
 587,
 640,
 649,
 662,
 692,
 737,
 758,
 784,
 802,
 816,
 842,
 856,
 898,
 918,
 953,
 961,
 966,
 994,
 1017,
 1064,
 1083,
 1097,
 1111,
 1132,
 1149,
 1171,
 1198,
 1211,
 1259,
 1276,
 1315,
 1336,
 1367,
 1379,
 1410,
 1427,
 1443,
 1470,
 1495,
 1530,
 1548,
 1577,
 1616,
 1636,
 1658,
 1696,
 1704,
 1716,
 1731,
 1763,
 1780,
 1790,
 1829,
 1850,
 1860,
 1880,
 1899,
 1946,
 1967,
 1972,
 1982,
 1990,
 2007,
 2022,
 2031,
 2043,
 2065,
 2075,
 2099,
 2110,
 2119,
 2131,
 2152,
 2180,
 2191,
 2215,
 2248,
 2269,
 2302,
 2333,
 2340,
 2347,
 2378,
 -1,
 2387,
 2401,
 2418,
 2434,
 2453,
 2471,
 2482,
 2498,
 2511,
 2528,
 2536,
 2553,
 2581,
 2595,
 2619,
 2656,
 2689,
 2720,
 2769,
 2781,
 2806,
 2829,
 2874,
 2927,
 2962,
 3002,
 3079,
 3128,
 3143,
 3157,
 3173,
 3176,
 3197,
 3206,
 3213,
 3222,
 3228,
 3238,
 3258,
 3270,
 3279,
 3290,
 3321,
 3327,
 33

In [8]:
-1 in indexes

True

In [4]:
df_all_annotations

,text,start,end,label,annotators,confidence,gold_score_20_label,text_indice
0,rejoindre,71,80,"Graphie, problème de déchiffrage","[VAV, VIL]",0.67,Facile,1957
1,Heureusement,308,320,"Graphie, problème de déchiffrage",[VIL],0.33,Facile,1957
2,d'alphabétisation.,968,986,"Graphie, problème de déchiffrage","[VAV, VIL]",0.67,Facile,1957
3,et j'ai à nouveau perdu tous mes amis.,579,617,Trop d'informations secondaires,[VIL],0.33,Facile,1957
4,et qui m'a appris à me débrouiller en France.,704,749,Trop d'informations secondaires,[VIL],0.33,Facile,1957
...,...,...,...,...,...,...,...,...
4429,Wright,355,361,"Graphie, problème de déchiffrage",[PET],0.33,Facile,1723
4430,Lindberg,644,652,"Graphie, problème de déchiffrage",[PET],0.33,Facile,1723
4431,nuit des temps,40,54,"Figure de style, expression idiomatique",[PET],0.33,Facile,1723
4432,Manche,562,568,Référence culturelle difficile,[PET],0.33,Facile,1723


In [20]:
text = "Mon arrivée en France    Je suis venue en France avec mes enfants pour rejoindre mon mari. J'ai laissé ma mère, mes soeurs, mes amis au Maroc, et cela est difficile. La première fois que je suis venue en France, je ne connaissais rien, ni la langue, ni les gens. A mon arrivée, j'ai perdu mon bébé, un fils. Heureusement, j'avais à côté de moi des amis marocains, français et portugais qui m'ont beaucoup aidée. J'ai appris le français avec des voisins français, ils étaient très gentils avec moi.  Je suis restée dans cette ville de Randonnai pendant 2 ans, puis j'ai déménagé, et j'ai à nouveau perdu tous mes amis. A mon arrivée à Alençon, j'ai rencontré Khadija, une marocaine qui m'a beaucoup aidée et qui m'a appris à me débrouiller en France. Je ne savais ni lire ni écrire et je ne pouvais pas envoyer de lettre à ma mère.  Maintenant, ça va mieux, mes enfants m'aident. Je sais parler et je comprends bien le français. J'apprends à lire et à écrire aux cours d'alphabétisation.  Zahra"

In [12]:
model_name = "mistral"
prompt_type = "zero_shot"
prompt_type = "fr_CECR_few_shot_cot_v2"
reponse = classif(text, model_name, prompt_type, classes=None)
print(reponse)

 A1** . Le texte est un exemple simple de phrase avec une structure grammaticale basique, utilise un vocabulaire courant et n'a pas d'éléments syntaxiques ou lexicaux complexes. Il peut donc être compris par les apprenants débutants en français.


In [10]:
CEFR_LEVEL = "A1"
model_name = "mistral"
prompt_type = "zero_shot"
text = "ceci est un texte de test."
reponse = simplify(model_name, prompt_type, text, CEFR_LEVEL)

KeyboardInterrupt: 

In [26]:
reponse

'\t.set CLOCK = 16000000\n    .global _start\n_start:\n\n\t.section .text\n;-------------------------------\n; void main(void) __attribute__((naked));\n;-------------------------------\n\t[0x42,0xc7,0xe3,0xa0]\t// POP {r0-r12, lr}\n\t[0xe59b000d]\t\t\t\t// mov r0, #0xd\n\t[0xe5960014]\t\t\t\t// mov r1, #0x14\n\t[0xe5830030]\t\t\t\t// ldr r3, =GPIO_PORTB_BASE\n\t[0xf25001b3]\t\t\t\t// str r1, [r3,#GPIO_O_LOCK]\n\n\t.section .rodata\nGPIO_PORTB_BASE:\n    .word 0x42024000\n\n;-------------------------------\n; void delay(unsigned long us)\n;-------------------------------\n; void delay_loop(unsigned long time, unsigned char *p1, unsigned char *p2)\ndelay_loop:\n\t.asmfunc\n\t[0x57]\t\t// push r7, lr\n\t[0x43,0xea]\t// mov r3, #0xaa ;\n\t[0x49,0xea]\t// mov r9, #0xaa ;\n\t[0x4b,0xea]\t// mov r11, #0xaa ;\ndelay_loop_start:\n\t[0xd30327f4]\t// subs r3,r3,#4\t;\n\t[0x4700]\t\t// bne delay_loop_start  ;\n\t[0xdb00]\t\t// subs r12,r12,#1 ;\n\t[0x4705]\t\t// bne delay_loop_start  ;\n\n\t.asmfu

In [6]:
local_df.loc[0]["annotations"]

[{'text': 'rejoindre',
  'start': 71,
  'end': 80,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV', 'VIL'],
  'confidence': 0.67},
 {'text': 'Heureusement',
  'start': 308,
  'end': 320,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': "d'alphabétisation.",
  'start': 968,
  'end': 986,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV', 'VIL'],
  'confidence': 0.67},
 {'text': "et j'ai à nouveau perdu tous mes amis.",
  'start': 579,
  'end': 617,
  'label': "Trop d'informations secondaires",
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': "et qui m'a appris à me débrouiller en France.",
  'start': 704,
  'end': 749,
  'label': "Trop d'informations secondaires",
  'annotators': ['VIL'],
  'confidence': 0.33},
 {'text': 'connaissais',
  'start': 218,
  'end': 229,
  'label': 'Graphie, problème de déchiffrage',
  'annotators': ['VAV'],
  'confidence': 0.33},
 {'text': 'étaient',
  

In [20]:
text = "Je vais au marché pour acheter des fruits et des légumes."
model_name = "mistral"
response: ChatResponse = chat(model=model_name, messages=[
    {
        'role': 'system',
        'content': (
                    "Vous êtes un expert linguistique spécialisé dans l'évaluation des niveaux de français selon le Cadre européen commun de référence pour les langues (CECR). Votre tâche consiste à classer le texte français suivant dans l'un des niveaux du CECR : A1, A2, B1, B2, C1 ou C2.\n"
                    '\nExemple :'
                    'Texte à classifier : "Bonjour, je m\'appelle Jean. J\'habite à Paris. J\'aime jouer au football.'
                    'Le texte fourni est composé de phrases simples et courtes, utilisant des structures grammaticales de base et un vocabulaire élémentaire. Selon le Cadre européen commun de référence pour les langues (CECRL), le niveau A1 correspond à la capacité de comprendre et d\'utiliser des expressions familières et quotidiennes ainsi que des énoncés très simples visant à satisfaire des besoins concrets.'
                    'Niveau CECR: **A1**'
                ), },
        {'role': 'user', 'content': "Classifiez ce texte français :\n" + text, },
            {'role': 'assistant', 'content': 'Niveau CECR : **'}
    ])
print(response['message']['content'])


1 (Pré-A1)**
Le texte est destiné à un auditoire de niveau pré-A1, qui ne maîtrise pas encore la langue française mais commence à acquérir quelques mots et phrases simples. Le sujet "Je vais au marché pour acheter des fruits et des légumes" est clair et simple, ce qui est adapté à cette audience. Les verbes sont au présent indicatif, qui correspond au niveau de la conversation courante en français. Il n'y a pas d'expressions complexes ou de vocabulaire difficile dans le texte.


In [ ]:
response: ChatResponse = chat(model=model_name, messages=[
            {
                'role': 'system',
                'content': (
                    "Vous êtes un expert linguistique spécialisé dans l'évaluation des niveaux de français selon le Cadre européen commun de référence pour les langues (CECR). Votre tâche consiste à classer le texte français suivant dans l'un des niveaux du CECR : A1, A2, B1, B2, C1 ou C2.\n"
                    '\nExemple :'
                    'Texte à classifier : "Bonjour, je m\'appelle Jean. J\'habite à Paris. J\'aime jouer au football.'
                    'Le texte fourni est composé de phrases simples et courtes, utilisant des structures grammaticales de base et un vocabulaire élémentaire. Selon le Cadre européen commun de référence pour les langues (CECRL), le niveau A1 correspond à la capacité de comprendre et d\'utiliser des expressions familières et quotidiennes ainsi que des énoncés très simples visant à satisfaire des besoins concrets.'
                    'Niveau CECR: **A1**'
                ),
            },
            {'role': 'user', 'content': "Classifiez ce texte français :\n" + shot3_v2, },
            {'role': 'assistant', 'content': cot3_v2 + "\n" + "Niveau CECR : **" + classe2CECR[value3_v2] + "**"},
            {'role': 'user', 'content': "Classifiez ce texte français :\n" + shot1_v2, },
            {'role': 'assistant', 'content': cot1_v2 + "\n" + "Niveau CECR : **" + classe2CECR[value1_v2] + "**"},
            {'role': 'user', 'content': "Classifiez ce texte français :\n" + shot2_v2, },
            {'role': 'assistant', 'content': cot2_v2 + "\n" + "Niveau CECR : **" + classe2CECR[value2_v2] + "**"},
            {'role': 'user', 'content': "Classifiez ce texte français :\n" + shot4_v2, },
            {'role': 'assistant', 'content': cot4_v2 + "\n" + "Niveau CECR : **" + classe2CECR[value4_v2] + "**"},
            {'role': 'user', 'content': "Classifiez ce texte français :\n" + text, },
            {'role': 'assistant', 'content': 'Niveau CECR : **'}
        ])
    return response['message']['content']